In [2]:
import duckdb
con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/data/deadlock.db")



In [3]:
df = con.execute("SELECT * FROM player_matches LIMIT 10").fetchdf()
df

,account_id,match_id,hero_id,team,kills,deaths,assists,denies,net_worth
0,1146874782,35509898,14,Team0,1,5,11,5,34990
1,854939296,35509898,10,Team1,8,6,20,24,32167
2,78706118,35509898,35,Team0,1,6,7,7,23335
3,31357831,35509898,31,Team1,8,2,18,3,35845
4,200135833,35509898,50,Team0,2,9,10,2,28047
5,1030720219,35509898,19,Team1,12,1,14,2,38473
6,129093286,35509898,15,Team1,4,7,14,0,32979
7,158752514,35509898,4,Team1,11,2,18,11,45330
8,99844341,35509898,2,Team1,7,5,14,0,37577
9,193143480,35509898,13,Team0,4,5,6,12,35192


In [ ]:
response = con.execute("""SELECT 
    account_id,
    COUNT(DISTINCT match_id) AS match_count
FROM 
    player_matches
GROUP BY 
    account_id
ORDER BY 
    match_count DESC
LIMIT 25;""").fetchdf()
print(response)

In [ ]:
response = con.execute("""SELECT 
    match_count,
    COUNT(*) AS player_count
FROM (
    SELECT 
        account_id,
        COUNT(DISTINCT match_id) AS match_count
    FROM 
        player_matches
    GROUP BY 
        account_id
) t
GROUP BY 
    match_count
ORDER BY 
    match_count;""").fetchdf()
print(response)

In [ ]:
response = con.execute("""
        SELECT 
            pm.account_id,
            COUNT(DISTINCT pm.match_id) AS match_count,
            MAX(m.start_time) AS match_time
        FROM 
            player_matches pm
        JOIN
            matches m ON pm.match_id = m.match_id
        GROUP BY 
            pm.account_id
        HAVING 
            match_count > 10
        ORDER BY
            match_time DESC
        LIMIT 5
        """).fetchdf()
print(response)

In [ ]:
response = con.execute("""
        SELECT 
           for match_id in matches
            count(account_id) in player_trends
        """).fetchdf()
print(response)

In [6]:
df = con.execute("""
    SELECT *
    FROM player_trends
                 limit 25
""").fetchdf()
print(df)

    account_id  p_average_kills  p_average_deaths  p_avg_kd  p_total_matches  \
0    158752514              NaN               NaN       NaN             <NA>   
1     97506421              NaN               NaN       NaN             <NA>   
2   1558594524              NaN               NaN       NaN             <NA>   
3   1146769754              NaN               NaN       NaN             <NA>   
4   1032223691              NaN               NaN       NaN             <NA>   
5    358780740              NaN               NaN       NaN             <NA>   
6    381952778              NaN               NaN       NaN             <NA>   
7   1685634485              NaN               NaN       NaN             <NA>   
8    150454049              NaN               NaN       NaN             <NA>   
9    172423864              NaN               NaN       NaN             <NA>   
10   183362050              NaN               NaN       NaN             <NA>   
11   344336241              NaN         